# Limpieza y Normalización de los DataFrames
Vamos a limpiar y normalizar tanto los datos de entrenamiento 'train' como los de 'test'.

| | original data | normalized data |
|-------------------|---------------|-----------------|
| training data | dataset_train.csv | normal_train.csv |
| test data | dataset_test.csv | normal_test.csv 

Diferencias entre los datos de entrenamiento y de test originales:
- Los datos de entrenamiento tienen 1600 registros y los de test tienen 400 registros.
- Los datos de test tienen vacía la columna 'Hogwarts House' que es la columa objetivo de la clasificación.

Trabajaremos con una versión reducida (lite5) que contiene:
- Las características base:
    1. 'Best Hand'
    2. 'Age'
- Las 5 asignaturas principales:
    1. 'Defense Against the Dark Arts'
    2. 'Herbology'
    3. 'Potions'
    4. 'Charms'
    5. 'Flying'
- Para los datos de entrenamiento también se incluyen las variables dummy de 'Hogwarts House'. 

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from datetime import datetime

# Definición de las características que vamos a mantener
BASE_FEATURES = ['Best Hand', 'Age']
LITE5_COURSES = ['Defense Against the Dark Arts', 'Herbology',
                 'Potions', 'Charms', 'Flying'
]

## Lectura y preparación inicial de los datos

Leemos los datasets originales y calculamos la edad a partir de la columna Birthday.

In [ ]:
# Lectura de los datasets
file_train = '../datasets/dataset_train.csv'
file_test = '../datasets/dataset_test.csv'

df_train = pd.read_csv(file_train, index_col=0)
df_test = pd.read_csv(file_test, index_col=0)

## Imputación de datos faltantes usando 'Astronomy'
Usamos la correlación perfecta r=-1 entre 'Astronomy' y 'Defense Against the Dark Arts' para realizar la imputación de los datos faltantes que se puedan imputar de 'Defense Against the Dark Arts' aprovechando esta característica.

Lo único que hay que hacer es tomar el dato de 'Astronomy' y multiplicar por -0.01 para lograr conseguir el dato faltante de  'Defense Against the Dark Arts'.

In [ ]:
# Imputar 'Defense Against the Dark Arts' usando 'Astronomy'

def data_imputation_perfect_correlation(df):
    # Calcular las filas totales
    total = len(df)
    
    # Calcular las filas con datos no nulos antes de la imputación
    n1 = df['Defense Against the Dark Arts'].count()
    
    # Crear máscara para identificar:
    # 1. Valores nulos en Defense Against the Dark Arts
    # 2. Valores no nulos en Astronomy
    mask = (df['Defense Against the Dark Arts'].isna() & 
            df['Astronomy'].notna())
    
    # Imputar los valores usando la relación perfecta (Astronomy * -0.01)
    df.loc[mask, 'Defense Against the Dark Arts'] = df.loc[mask, 'Astronomy'] * -0.01
    
    # Calcular las filas con datos no nulos después de la imputación
    n2 = df['Defense Against the Dark Arts'].count()
    
    # Verificar el resultado
    print("Valores no nulos en Defense Against the Dark Arts:")
    print(f"\tAntes de imputación: {n1}/{total}")
    print(f"\tDespués de imputación: {n2}/{total}")
    
    print(f"\nHemos recuperado {n2-n1} filas.")

# Aplicación a los dos DataFrame
print(f"{'='*13} Para el DataFrame train {'='*13}")
data_imputation_perfect_correlation(df_train)
print()
print("="*13, "Para el DataFrame test", "="*13)
data_imputation_perfect_correlation(df_test)

In [ ]:
df_train.info()

In [ ]:
df_test.info()

## Cálculo de la edad
Es importante que la fecha que se tome como base (reference_date) para el cálculo de la edad en el DataFrame de entrenamiento se guarde en una variable para luego usarla como base en el cálculo de la edad en el DataFrame de test.

In [ ]:
# Primero, convertimos Birthday a datetime en ambos datasets
df_train['Birthday'] = pd.to_datetime(df_train['Birthday'])
df_test['Birthday'] = pd.to_datetime(df_test['Birthday'])

# Calculamos la fecha de referencia usando solo el conjunto de entrenamiento
reference_date = df_train['Birthday'].max()

# Definimos la función para calcular la edad
def calculate_age(df, reference_date):
    df['Age'] = (reference_date - df['Birthday']).dt.days / 365.25
    return df

# Aplicamos el cálculo de edad a ambos datasets
df_train = calculate_age(df_train, reference_date)
df_test = calculate_age(df_test, reference_date)

## Eliminación de columnas innecesarias
Incluyendo 'Astronomy' después de usarla para la imputación

In [ ]:
# Eliminamos las columnas que no necesitamos
columns_to_drop = ['First Name', 'Last Name', 'Birthday', 'Astronomy']
df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)

## Conversión de datos categóricos a numéricos
Convertimos 'Best Hand' a valores numéricos de tipo `float`.

In [ ]:
# Convertimos Best Hand a valores numéricos en ambos datasets
hand_mapping = {'Left': 0, 'Right': 1}
df_train['Best Hand'] = df_train['Best Hand'].map(hand_mapping)
df_test['Best Hand'] = df_test['Best Hand'].map(hand_mapping)

# Convertimos Best Hand a float en ambos datasets
df_train['Best Hand'] = df_train['Best Hand'].astype(float)
df_test['Best Hand'] = df_test['Best Hand'].astype(float)

# Verificamos la conversión
print("Valores únicos en Best Hand (train):", df_train['Best Hand'].unique())
print("Valores únicos en Best Hand (test):", df_test['Best Hand'].unique())

## Tratamiento específico para los datos de entrenamiento

Para los datos de entrenamiento necesitamos hacer one-hot encoding de la columna 'Hogwarts House'.

In [ ]:
# One-hot encoding solo para los datos de entrenamiento
df_train = pd.get_dummies(df_train, columns=['Hogwarts House'], prefix='House', dtype=float)

In [ ]:
# Actualizamos BASE_FEATURES para incluir las columnas dummy de House
HOUSE_FEATURES = ['House_Gryffindor', 'House_Hufflepuff', 'House_Ravenclaw', 'House_Slytherin']
BASE_FEATURES = BASE_FEATURES + HOUSE_FEATURES

## Selección de características
Seleccionamos solo las características de lite5 para luego normalizar las columnas numéricas.

In [ ]:
# Seleccionamos las columnas que queremos mantener
df_train = df_train[LITE5_COURSES + BASE_FEATURES]
df_test = df_test[LITE5_COURSES + ['Best Hand', 'Age']]  # Note que no incluimos HOUSE_FEATURES para test

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
df_train.head(124)

### Eliminación de las filas con datos faltantes en 'train'
- Se eliminan solo las filas de las características con las que se trabaja y solo del DataFrame 'df_train'.
- En el 'df_test' no se elimina ninguna fila ya que luego aplicaremos un algoritmo de imputación para completar los datos faltantes. 

In [ ]:
# Filas de los DataFrame previos a la eliminación de filas faltantes
n_train = len(df_train)

df_train = df_train.dropna()

print(f"El DataFrame 'train' ha pasado de {n_train} a {len(df_train)} filas.")
print(f"Se han eliminado {n_train - len(df_train)} registros con datos faltantes.")
print(f"Se han eliminado solo el {n_train / len(df_train) - 1:.1%} de los registros, por este motivo no realizamos imputación de datos en 'train'.")

In [ ]:
df_train.info()

In [ ]:
df_train.head(124)
# Observe que el registro de Index 121 se ha eliminada ya que 'Herbology' tenía 'NaN' 

# Normalización
* Realizaremos la normalización antes de la imputación con el método de los k-vecinos más próximos (KNN) ya que:
    - Es necesario normalizar las variables a una escala común antes de aplicar KNN.
    - KNN se basa en el cálculo de distancias entre observaciones. Si hay variables con escalas muy diferentes, las variables con valores más grandes dominarán el cálculo de la distancia, sesgando los resultados
* Es crucial usar EXACTAMENTE los mismos parámetros de normalización (media y desviación estándar) que se usen para los datos de entrenamiento cuando se aplique la normalización a los datos de test.
* No se deben calcular nuevos parámetros con los datos de test.
* Las variables dummy (también llamadas variables indicadoras o binarias) no deben normalizarse puesto que ya están en una escala fija (0 y 1),

### Función de Normalización

In [ ]:
# Función de normalización
def normalize(column):
    mean = column.mean()
    std = column.std()
    return (column - mean) / std

### Aplicación de la Normalización a ambos DataFrames

In [ ]:
# Separar las columnas a normalizar de las dummies
# 'Best Hand' no se normalizará´
features_to_normalize = ['Defense Against the Dark Arts', 'Herbology', 'Potions', 
                        'Charms', 'Flying', 'Age']

# Normalizar df_train (excluyendo las variables dummy)
for column in features_to_normalize:
    mean = df_train[column].mean()
    std = df_train[column].std()
    
    # Normalizar en train
    df_train[column] = (df_train[column] - mean) / std
    
    # Usar los MISMOS parámetros para normalizar test
    df_test[column] = (df_test[column] - mean) / std

# Las variables dummy (HOUSE_FEATURES) se mantienen sin normalizar

In [ ]:
# Guardado del Datasets normalizado de entrenamiento
df_train.to_csv('../datasets/normal_train.csv')
# El Dataset de test aún no se graba puesto que faltan las imputaciones a datos faltantes

print("Dimensiones del dataset de entrenamiento:", df_train.shape)
print("Dimensiones del dataset de test:", df_test.shape)
print("\nColumnas en el dataset de entrenamiento:\n\t", df_train.columns.tolist())
print("\nColumnas en el dataset de test:\n\t", df_test.columns.tolist())

**Nota**
- El Dataset de test contiene 400 registros pero aún contiene algunos registros con datos faltantes.
- Aún tenemos que realizar la imputación con la técnica de los k-vecinos más próximos (KNN) [k-nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)

# Imputación de datos faltantes en el Dataset 'test'
Son pocos los registros que tienen algún dato faltante en el Dataset de `test`pero necesitamos que esten completos los 400 regitros para poder finalmente grabar el archivo 'houses.csv' que contendrá el resultado de nuestro entrenamiento aplicado a los datos de test. Es necesario que sean 400 registros por si se evalúan todos y cada uno de ellos confrontando nuestra estimación de la clasificación son los datos reales que nosotros no tenemos.

In [ ]:
# Contar filas que tienen AL MENOS UN valor faltante
n_rows_with_na = df_test.isna().any(axis=1).sum()
print(f"Filas con al menos un dato faltante en 'df_test': {n_rows_with_na}.")

## Imputación de datos faltantes con KNN usando solo filas completas

### 1. Preparación de los datos:

In [ ]:
# Identificar filas completas (sin valores faltantes)
complete_rows = df_test.dropna()

# Identificar filas con valores faltantes
incomplete_rows = df_test[df_test.isnull().any(axis=1)]

### 2. Función para calcular distancia euclidiana:

In [ ]:
import numpy as np

def euclidean_distance(row1, row2, columns):
    """
    Calcula la distancia euclidiana entre dos filas usando solo las columnas especificadas
    """
    return np.sqrt(sum((row1[columns] - row2[columns])**2))

### 3. Función para encontrar k vecinos más cercanos:

In [ ]:
def get_k_neighbors(target_row, complete_data, columns_to_use, k=5):
    """
    Encuentra los k vecinos más cercanos para una fila dada
    """
    distances = []
    for idx, row in complete_data.iterrows():
        dist = euclidean_distance(target_row, row, columns_to_use)
        distances.append((idx, dist))

    # Ordenar por distancia y obtener los k más cercanos
    distances.sort(key=lambda x: x[1])
    return [idx for idx, _ in distances[:k]]

### 4. Función de imputación:

In [ ]:
def impute_missing_values(df, k=5):
    """
    Imputa valores faltantes usando KNN
    """
    df_imputed = df.copy()
    complete_data = df.dropna()

    # Para cada fila con valores faltantes
    for idx, row in df[df.isnull().any(axis=1)].iterrows():
        # Identificar columnas con valores faltantes en esta fila
        missing_cols = row[row.isnull()].index

        # Identificar columnas disponibles para calcular distancias
        available_cols = row[row.notnull()].index

        # Encontrar k vecinos más cercanos
        neighbors_idx = get_k_neighbors(row, complete_data, available_cols, k)

        # Imputar cada columna faltante con la media de los vecinos
        for col in missing_cols:
            neighbor_values = complete_data.loc[neighbors_idx, col]
            df_imputed.loc[idx, col] = neighbor_values.mean()

    return df_imputed

### 5. Aplicación del proceso:

In [ ]:
# Aplicar la imputación
df_test_imputed = impute_missing_values(df_test, k=5)

# Verificar resultados
print("\nVerificación de valores faltantes después de imputación:")
print(df_test_imputed.isnull().sum())

### Grabación del Dataset de 'test' normalizado y sin datos faltantes

In [ ]:
df_test_imputed.to_csv('../datasets/normal_test.csv')